## TERM-PROJECT: 주제에 따른 텍스트 분류 시스템
### 경희대학교 소프트웨어융합학과 김민서

In [ ]:
pip install konlpy

In [ ]:
pip install gensim nltk

In [ ]:
!pip install pyLDAvis

In [1]:
import pandas as pd
import re
from konlpy.tag import Okt

# 1. 사용자 파일 입력
file_name = input("업로드할 파일의 이름을 입력하세요: ")
file_name='./'+file_name+'.csv'
column_name = input("데이터가 있는 칼럼의 이름을 입력하세요: ")
    
# 2. 파일 부분 확인
try:
    df_news_all = pd.read_csv(file_name, encoding='cp949')
    article_titles = df_news_all[column_name].tolist()
except FileNotFoundError:
    print ('error: 파일을 찾을 수 없습니다.')
except KeyError:
    print ('error: 칼럼이 존재하지 않습니다.')

# 3. 데이터 전처리

# 3-1. 한글과 공백을 제외만 모든 문자를 빈 문자열로 대체
article_titles = [re.sub(r'[^가-힣\s]','', title) for title in article_titles if type(title) != float]

# 3-2. 토큰화하기
okt = Okt()
article_titles_tokenized = [okt.morphs(title) for title in article_titles]

# 3-3. 불용어 제거하기
korean_stopwords = ['은', '는', '이', '가', '을', '를', '으로', '에서', '과', '와', '의', '에', '하다']
article_titles_filtered = [[word for word in title if word not in korean_stopwords] for title in article_titles_tokenized]
    
# 3-4. 품사 태깅하기
from konlpy.tag import Okt
pos_tags = []
# 만약, 중첩리스트로 형성이 된다면, 중첩 리스트에 대해서도 읽을 수 있도록 코드 작성
first_element = article_titles_filtered[0]
if type(first_element)==list:
    for sublist in article_titles_filtered:
        for title in sublist:
            if isinstance(title, str):
                pos_tags.append(okt.pos(title))
            else:
                print("Error: title is not a string", title)

# 3-5. 특성 추출하기
# 카운트 벡터라이제이션 사용
from sklearn.feature_extraction.text import CountVectorizer
tagged_texts = [' '.join([word + "/" + tag for word, tag in tagged_sentence]) for tagged_sentence in pos_tags]
count_vectorizer = CountVectorizer() # 카운트 벡터라이제이션 초기화
count_matrix = count_vectorizer.fit_transform(tagged_texts)
# TF-IDF 벡터라이제이션 사용
from sklearn.feature_extraction.text import TfidfVectorizer
tagged_texts = [' '.join([word+"/"+tag for word, tag in tagged_sentence]) for tagged_sentence in pos_tags]
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(tagged_texts)

# 4. 비지도 학습 - 토픽 모델링

# 4-1. LDA 모델 구축
import gensim
from gensim import corpora
# 딕셔너리 생성
dictionary = corpora.Dictionary(article_titles_filtered)
# 코퍼스(corpus) 생성: 각 문서를 (단어 ID, 단어 빈도수) 형태로 변환
corpus = [dictionary.doc2bow(text) for text in article_titles_filtered]

# 4-2. LDA 모델 학습
lda_model = gensim.models.ldamodel.LdaModel(corpus, num_topics=30, alpha=0.01, eta=0.01, id2word=dictionary, passes=15)

# 4-3. 결과 출력: 각 토픽의 상위 단어들
for idx, topic in lda_model.print_topics(-1):
    print(f"{idx+1}번째 TOPIC \nWords: {topic}")

# 5. 모델 평가

from gensim.models.coherencemodel import CoherenceModel

# 5-1. Coherence Model 생성
coherence_model_lda = CoherenceModel(model=lda_model, texts=article_titles_filtered, dictionary=dictionary, coherence='c_v')

# 5-2. Coherence Score 계산
coherence_score = coherence_model_lda.get_coherence()
print('Coherence Score:', coherence_score)

업로드할 파일의 이름을 입력하세요: 부산광역시 부산진구_뉴스기사 테이블_20230725
데이터가 있는 칼럼의 이름을 입력하세요: 제목
1번째 TOPIC 
Words: 0.118*"안내" + 0.053*"명" + 0.051*"부산진구" + 0.048*"차" + 0.042*"선정" + 0.030*"작품" + 0.030*"가정" + 0.028*"온" + 0.026*"사진" + 0.024*"백양산"
2번째 TOPIC 
Words: 0.254*"부산" + 0.072*"지" + 0.067*"기탁" + 0.036*"쌀" + 0.027*"금연" + 0.025*"조사" + 0.022*"준비" + 0.021*"마음" + 0.020*"칼럼" + 0.019*"개설"
3번째 TOPIC 
Words: 0.085*"이웃" + 0.071*"희망" + 0.069*"안전" + 0.056*"돕기" + 0.056*"성" + 0.036*"쓰레기" + 0.028*"금품" + 0.027*"고맙습니다" + 0.026*"사랑" + 0.023*"비"
4번째 TOPIC 
Words: 0.080*"시" + 0.047*"대회" + 0.044*"도서관" + 0.037*"부산진구" + 0.035*"공모" + 0.033*"작은" + 0.028*"연" + 0.028*"공" + 0.028*"신발" + 0.027*"걷기"
5번째 TOPIC 
Words: 0.161*"운영" + 0.089*"의회" + 0.054*"부산진구" + 0.039*"인" + 0.036*"당감동" + 0.030*"여성" + 0.030*"잔치" + 0.030*"부암동" + 0.029*"교실" + 0.027*"첫"
6번째 TOPIC 
Words: 0.103*"등" + 0.069*"자" + 0.061*"안" + 0.057*"심사" + 0.053*"내" + 0.048*"조례" + 0.032*"관리" + 0.028*"아이" + 0.020*"실시" + 0.019*"없는"
7번째 TOPIC 
Words: 0.100*"센터" + 0.092*"주민" + 0.065*"

In [2]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, dictionary)
pyLDAvis.display(vis)